In [111]:
import os
import pdfplumber
from collections import defaultdict

def detect_columns(text_blocks, page_width):
    """Gruppiere Textblöcke basierend auf ihrer horizontalen Position (Spalten)."""
    columns = defaultdict(list)
    threshold = page_width * 0.15  # Threshold, um Spalten zu erkennen

    for block in text_blocks:
        x_center = (block['x0'] + block['x1']) / 2

        assigned = False
        for col_x in columns:
            if abs(col_x - x_center) < threshold:
                columns[col_x].append(block)
                assigned = True
                break
        
        if not assigned:
            columns[x_center].append(block)

    return columns

def extract_text_from_column(column_blocks):
    """Sortiere die Blöcke innerhalb einer Spalte und extrahiere den Text."""
    sorted_blocks = sorted(column_blocks, key=lambda block: (block['top'], block['x0']))
    return ' '.join([block['text'] for block in sorted_blocks])

def extract_text_from_page_g1(page):
    """Extrahiere den Text aus einer Seite für PDFs in g1, behalte dabei das Layout."""
    text_blocks = page.extract_words()
    if not text_blocks:
        return ""

    columns = detect_columns(text_blocks, page.width)
    full_text = []

    for col_x, blocks in sorted(columns.items()):
        column_text = extract_text_from_column(blocks)
        full_text.append(column_text)

    return "\n\n".join(full_text)

def extract_text_from_page_g2(page):
    """Alternative Methode zur Textextraktion für PDFs in g2."""
    # Beispielhafte einfache Extraktion, die besser auf das Layout abgestimmt ist
    # Dies kann durch Methoden wie pdfminer.six, die textorientierter sind, ergänzt werden.
    text_blocks = page.extract_words()
    if not text_blocks:
        return ""
    
    full_text = [block['text'] for block in sorted(text_blocks, key=lambda block: (block['top'], block['x0']))]
    
    return ' '.join(full_text)

def extract_text_from_pdfs(pdf_folder_path, extraction_function):
    documents = []

    for filename in os.listdir(pdf_folder_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder_path, filename)

            with pdfplumber.open(pdf_path) as pdf:
                full_text = []

                for page in pdf.pages:
                    page_text = extraction_function(page)
                    full_text.append(page_text)

                documents.append("\n\n".join(full_text))

    return documents

# Pfade zu den PDF-Ordnern
pdf_folder_path_g1 = os.path.join('data', 'pdf', 'g1')
pdf_folder_path_g2 = os.path.join('data', 'pdf', 'g2')

# Extrahiere die Texte aus den PDFs in g1
documents_g1 = extract_text_from_pdfs(pdf_folder_path_g1, extract_text_from_page_g1)

# Extrahiere die Texte aus den PDFs in g2
documents_g2 = extract_text_from_pdfs(pdf_folder_path_g2, extract_text_from_page_g2)

# Kombiniere alle Dokumente
documents = documents_g1 + documents_g2

# Beispielhafte Ausgabe des ersten Dokuments
# print(documents[2] if documents else "Keine Dokumente gefunden")
for i, doc in enumerate(documents):
    print(f"Document {i}:\n")
    print(doc)
    print("\n" + "="*50 + "\n")

Document 0:

Der blaue Altglascontainer. Das kommt hinein Saubere Flaschen und Deckelgläser nach den Farben Weiß, Grün und Braun getrennt; blaues oder buntes Glas bitte zum Grünglas geben.  Glasfl aschen  Konservengläser Das kommt nicht hinein  Flachglas wie Spiegel und Scheiben (Restmüll oder  Trinkgläser und Blumenvasen (Restmüll)  Porzellan und Keramik (Restmüll)  Glühbirnen (Restmüll), Energiesparlampen (Schadstoffmobil oder Wertstoffhöfe)  Stark verschmutzte Gläser (Restmüll) Standort: auf öffentlichen

immer Sperrmüll) Plätzen

Was kommt wo hinein? FES Frankfurter Entsorgungs- Abfall richtig trennen – und Service GmbH Weidenbornstraße 40 60389 Frankfurt am Main Servicetelefon 0800 2008007-0 Servicetelefax 069 212-31323 services@fes-frankfurt.de www.fes-frankfurt.de FES-Servicecenter Liebfrauenberg 52–54 (Ecke Töngesgasse/Schärfengässchen) 60313 Frankfurt am Main Mo. – Fr. 10.00 – 18.00 Uhr Sa. 10.00 – 16.00 Uhr rgungsfachbetrie Entso b rtschafte.V.-EdDn oE El -öK tsn 0 gla

In [112]:
import re

def clean_text(documents):
    """Durchläuft die Liste der Dokumente und bereinigt die Texte."""
    cleaned_documents = []
    
    # Liste unerwünschter Zeichen oder Muster
    unwanted_patterns = [
        r"",     # Entferne das Symbol 
        r"",     # Beispiel für ein weiteres unerwünschtes Symbol
        r"(\b\w+\s*-\s*\w+\b)",  # Entferne Worte mit unvollständigen Endungen (z.B. "Glasfl aschen")
        r"\n+",   # Entferne überflüssige Zeilenumbrüche
        r"\s{2,}",# Entferne überflüssige Leerzeichen
        r"\s*:\s*",  # Entferne überflüssige Doppelpunkte
        r"\b\w{1,2}\b",  # Entferne isolierte kurze Wörter (z.B. "fi" in "finden")
        r"[^A-Za-z0-9äöüÄÖÜß.,;:!?()\-'\s]"  # Entferne nicht-alphanumerische Zeichen
    ]

    # Regex-Muster kombinieren
    combined_pattern = re.compile("|".join(unwanted_patterns))

    for document in documents:
        # Entferne unerwünschte Zeichen und Muster
        cleaned_text = combined_pattern.sub(" ", document)
        # Entferne mehrfach aufeinanderfolgende Leerzeichen, um den Text zu bereinigen
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
        cleaned_documents.append(cleaned_text)
    
    return cleaned_documents

# Verwende die bestehende documents-Liste aus dem vorherigen Skript
cleaned_documents = clean_text(documents)

# Beispielhafte Ausgabe eines bereinigten Dokuments
# print(cleaned_documents[2])
for i, doc in enumerate(cleaned_documents):
    print(f"Document {i}:\n")
    print(doc)
    print("\n" + "="*50 + "\n")

Document 0:

Der blaue Altglascontainer. Das kommt hinein Saubere Flaschen und Deckelgläser nach den Farben Weiß, Grün und Braun getrennt; blaues oder buntes Glas bitte zum Grünglas geben. Glasfl aschen Konservengläser Das kommt nicht hinein Flachglas wie Spiegel und Scheiben (Restmüll oder Trinkgläser und Blumenvasen (Restmüll) Porzellan und Keramik (Restmüll) Glühbirnen (Restmüll), Energiesparlampen (Schadstoffmobil oder Wertstoffhöfe) Stark verschmutzte Gläser (Restmüll) Standort auf öffentlichen immer Sperrmüll) Plätzen Was kommt hinein? FES Frankfurter richtig trennen und Service GmbH Weidenbornstraße 60389 Frankfurt Main Servicetelefon 0800 Servicetelefax 069 services . www. . Liebfrauenberg (Ecke Töngesgasse Schärfengässchen) 60313 Frankfurt Main . . . . Uhr . . . Uhr rgungsfachbetrie Entso rtschafte. .-EdDn tsn gla3 sn1 we4 rgergemeinschaftZ efüR rrT DEI niF tmA chT hae enl , ElB rAe , Richtig trennen leicht gemacht. jedem Haus Frankfurt gehören Abfallbehälter Tonnen, Inhalt al

In [114]:
import re
import spacy

# Lade das deutsche spaCy-Modell
nlp = spacy.load('de_core_news_sm')

def preprocess_text_with_advanced_cleaning(documents):
    """Bereinigt die Dokumente, entfernt unnötige Stopwörter und normalisiert den Text."""
    custom_stopwords = nlp.Defaults.stop_words.difference({'nicht', 'kein', 'wichtig', 'ohne'})

    preprocessed_documents = []

    for document in documents:
        text = document.lower()
        text = re.sub(r'\b(?:[\w\-]+\.)+[\w]{2,4}(?:/[\w\-.?%&=]*)?\b', '', text)
        text = re.sub(r'\b[\w\.\-]+@[\w\.\-]+\.\w{2,4}\b', '', text)
        text = re.sub(r'\b(?:www|servicetelefon|servicetelefax|services|frankfurt|main|weidenbornstraße|liebfrauenberg)\b', '', text)
        text = re.sub(r'[^a-zäöüß\s]', '', text)

        doc = nlp(text)
        filtered_tokens = [token.text for token in doc if token.text not in custom_stopwords and len(token.text) > 2]

        # Entferne unzusammenhängende Zeichenfolgen und extrem kurze Wörter
        text = ' '.join(filtered_tokens)
        text = re.sub(r'\b\w{1,3}\b', '', text)

        # Entferne mehrfach aufeinanderfolgende gleiche Wörter
        text = re.sub(r'\b(\w+)\s+\1\b', r'\1', text)

        # Entferne weiterhin sinnlose Textfragmente
        text = re.sub(r'\b[a-zäöüß]{2,3}\b\s+\b[a-zäöüß]{2,3}\b', '', text)

        preprocessed_documents.append(text.strip())

    return preprocessed_documents

# Bereinige die Dokumente weiter
preprocessed_documents = preprocess_text_with_advanced_cleaning(cleaned_documents)

# Beispielhafte Ausgabe eines bereinigten Dokuments
# print(preprocessed_documents[2])
for i, doc in enumerate(preprocessed_documents):
    print(f"Document {i}:\n")
    print(doc)
    print("\n" + "="*50 + "\n")

Document 0:

blaue altglascontainer hinein saubere flaschen deckelgläser farben weiß grün braun getrennt blaues buntes glas bitte grünglas geben glasfl aschen konservengläser nicht hinein flachglas spiegel scheiben restmüll trinkgläser blumenvasen restmüll porzellan keramik restmüll glühbirnen restmüll energiesparlampen schadstoffmobil wertstoffhöfe stark verschmutzte gläser restmüll standort öffentlichen sperrmüll plätzen hinein  frankfurter trennen service gmbh               ecke töngesgasse schärfengässchen             rgungsfachbetrie entso rtschafte eddn   rgergemeinschaftz efür          trennen haus gehören abfallbehälter tonnen inhalt wertvoller rohstoff produkten weiterverarbeitet graue restmülltonne gelbe tonne verpackungen grüne tonne papier pappe braune tonne bioabfälle graue tonne restlichen abfall restmüll hinaus folgende entsorgungsangebote wertstoffe sichern altglas altglascontainer flaschen deckelgläser nden öffentlichen plätzen altkleider container altkleider schuhe nd

In [120]:
import spacy

# Lade das deutsche Sprachmodell
nlp = spacy.load('de_core_news_sm')

# Wörterbuch zur Korrektur von Lemmata und POS-Tags
manual_corrections = {
    "deckelgläser": ("NOUN", "Deckelglas"),
    "konservengläser": ("NOUN", "Konservenglas"),
    "keramik": ("NOUN", "Keramik"),
    "glühbirnen": ("NOUN", "Glühbirne"),
    "energiesparlampen": ("NOUN", "Energiesparlampe"),
    "weiß": ("ADJ", "weiß"),  # Spezifische Regel für das Adjektiv "weiß"
    "lacke": ("NOUN", "Lack"),  # Korrektur für Lacke als Nomen
    "wertstoffhöfe": ("NOUN", "Wertstoffhof"),  # Korrektur für Wertstoffhöfe
}

def correct_token(token):
    # Falls eine manuelle Korrektur vorliegt, diese anwenden
    if token.text.lower() in manual_corrections:
        return manual_corrections[token.text.lower()]
    else:
        lemma = token.lemma_
        pos = token.pos_
        return pos, lemma

def process_document(doc):
    spacy_doc = nlp(doc)
    
    processed_tokens = []
    for token in spacy_doc:
        # Korrekturen für häufige Fehler und Lemmatisierung
        pos, lemma = correct_token(token)

        # Nur relevante POS-Tags speichern (z.B. Noun, Verb, Adj)
        if pos in {"NOUN", "VERB", "ADJ"}:
            processed_tokens.append((token.text, pos, lemma))
    
    return processed_tokens

def process_documents(documents):
    return [process_document(doc) for doc in documents]

# Beispielnutzung:
processed_documents = process_documents(preprocessed_documents)

# Ausgabe des Ergebnisses für das erste Dokument
for token_info in processed_documents[0]:
    print(token_info)

('blaue', 'ADJ', 'blau')
('altglascontainer', 'NOUN', 'Altglascontainer')
('saubere', 'ADJ', 'sauber')
('flaschen', 'NOUN', 'Flasche')
('deckelgläser', 'NOUN', 'Deckelglas')
('farben', 'NOUN', 'Farbe')
('weiß', 'ADJ', 'weiß')
('getrennt', 'VERB', 'trennen')
('blaues', 'ADJ', 'blau')
('buntes', 'ADJ', 'bunt')
('glas', 'NOUN', 'Glas')
('bitte', 'ADJ', 'bitte')
('grünglas', 'NOUN', 'Grünglas')
('geben', 'VERB', 'geben')
('aschen', 'NOUN', 'Aschen')
('konservengläser', 'NOUN', 'Konservenglas')
('scheiben', 'NOUN', 'Scheiben')
('trinkgläser', 'NOUN', 'Trinkgläser')
('blumenvasen', 'NOUN', 'blumenvasen')
('restmüll', 'ADJ', 'Restmüll')
('keramik', 'NOUN', 'Keramik')
('glühbirnen', 'NOUN', 'Glühbirne')
('restmüll', 'NOUN', 'Restmüll')
('energiesparlampen', 'NOUN', 'Energiesparlampe')
('wertstoffhöfe', 'NOUN', 'Wertstoffhof')
('verschmutzte', 'ADJ', 'verschmutzt')
('gläser', 'ADJ', 'Gläser')
('restmüll', 'ADJ', 'Restmüll')
('standort', 'NOUN', 'Standort')
('öffentlichen', 'ADJ', 'öffentlich')


In [124]:
import os
import pickle

def save_processed_documents(processed_documents, filename="processed_documents.pkl"):
    # Sicherstellen, dass der Ordner "data/pickle" existiert
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    # Speichern der Datei im angegebenen Pfad
    with open(filename, "wb") as file:
        pickle.dump(processed_documents, file)
    
    print(f"Die Datei '{filename}' wurde erfolgreich gespeichert.")

# Beispielnutzung:
save_processed_documents(processed_documents, filename="data/pickle/processed_documents.pkl")

Die Datei 'data/pickle/processed_documents.pkl' wurde erfolgreich gespeichert.
